In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [29]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [3]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [4]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [5]:
dict_list = []
dummy = 0

for film_id in film_ids[46000:47000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

TimeoutException: Message: timeout
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.13.2 x86_64)


In [6]:
film_ids[46371]

'tt4481514'

In [7]:
dummy = 371

for film_id in film_ids[46371:47000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491


TimeoutException: Message: timeout
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.13.2 x86_64)


In [17]:
dummy = 492

for film_id in film_ids[46492:47000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630


TimeoutException: Message: timeout
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.13.2 x86_64)


In [18]:
dummy = 630

for film_id in film_ids[46630:47000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879


In [32]:
#491 - Dunkirk

In [43]:
Dunkirk = {'actors': '+_+Fionn Whitehead+_+Damien Bonnard+_+Aneurin Barnard+_+Lee Armstrong+_+James Bloor+_+Barry Keoghan+_+Mark Rylance+_+Tom Glynn-Carney+_+Tom Hardy+_+Jack Lowden+_+Luke Thompson+_+Michel Biel+_+Constantin Balsan+_+Billy Howle+_+Mikey Collins',
 'budget': '$100,000,000\n            ',
 'countries': '+_+UK+_+Netherlands+_+France+_+USA', 
 'directors': '+_+Christopher Nolan',
 'genres': '+_+Action+_+Drama+_+History',
 'gross_usa': ' $188,373,161, ',
 'keywords': '+_+world war two+_+soldier+_+evacuation+_+1940s+_+army',
 'languages': '+_+English+_+French+_+German',
 'mpaa': 'PG-13',
 'rating': '8.1',
 'release_date': '2017-07-21',
 'runtime': 106,
 'title': 'Dunkirk (2017)',
 'votes': '329,867',
 'writers': "+_+Christopher Nolan",
 'year': '2017'}

In [ ]:
#Dunkirk - soldie

In [55]:
dict_list.append(Dunkirk) #491 - Dunkirk

In [58]:
len(dict_list)

1000

In [59]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_47.json","w") as f:
      f.write(data)

with open("film_data_47.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Bella Thorne+_+Chandler Riggs+_+Jared Abrah...,"$5,000,000\n",+_+USA,+_+Sean Carter,+_+Horror+_+Thriller,None,+_+hidden camera+_+serial killers+_+home invasion,+_+English,R,5.4,2017-10-31,NaN,Keep Watching (2017),267,+_+Joseph Dembner,2017
1,+_+Rebecca Spence+_+Jessie Pinnick+_+Malic Whi...,None,+_+USA,+_+Stephen Cone,+_+Drama+_+Romance,None,+_+lesbian kiss+_+lesbian interest+_+friends w...,+_+English,None,6.5,2017-11-03,96.0,Princess Cyd (2017),469,+_+Stephen Cone,2017
2,+_+Tequan Richmond+_+Jenna Boyd+_+Christina Sc...,None,+_+USA,+_+G. Robert Vornkahl,+_+Crime+_+Thriller,None,,+_+English,None,6.8,2017-05-04,NaN,"Nowhere, Michigan (2017)",6,+_+Andrew Beguin,2017
3,+_+Jessica Partridge+_+Christopher Partridge+_...,None,+_+USA,+_+Nigel Bach,+_+Horror,None,,+_+English,None,4.1,2017-05-04,94.0,Steelmanville Road (2017),64,+_+Nigel Bach,2017
4,+_+Tandi Tugwell+_+Jenae Altschwager+_+Juanita...,None,+_+USA,+_+Josh Mitchell,+_+Drama,None,,+_+English,None,8.5,2017-05-04,90.0,Wine Tasting (2017),12,+_+Justin Samuels,2017
5,+_+Kyle Marasciulo+_+Da Ton+_+Ioan Johnstone-R...,None,+_+USA,"+_+Frank Delgado,+_+David Hutcherson",+_+Adventure,None,,+_+English,None,None,2017-05-04,NaN,The Road Runner (2017),None,"+_+Nicholas Martinez,+_+Nicky Michaeli",2017
6,+_+Kristin Slaysman+_+Scott Rodgers+_+Ashley S...,None,+_+USA,+_+Josh Crockett,+_+Comedy+_+Drama,None,,+_+English,None,6.6,2017-05-04,94.0,Dr. Brinks & Dr. Brinks (2017),7,"+_+Josh Crockett,+_+Jonathan Pappas",2017
7,+_+Sophie Bauer+_+Scott Brault+_+Ernest Briggs...,None,+_+USA,+_+Daniel X. Goate,+_+Action,None,,+_+English,None,None,2017-05-04,NaN,Area of Conflict (2017),None,+_+Si Dunn+_+Dan Gagliasso,2017
8,+_+Tiffany Browne-Tavarez+_+Mickey Deymon+_+Na...,"$20,000\n",+_+USA,"+_+Shalom Kolontarov,+_+Yakov Kolontarov",+_+Horror,None,+_+love+_+trailer+_+superhero+_+exploration+_+...,+_+English,None,None,2017-05-04,NaN,The Book of Secrets (2017),None,+_+Shalom Kolontarov,2017
9,+_+Jeff Garlin+_+Natasha Lyonne+_+Ava Acres+_+...,None,+_+USA,+_+Jeff Garlin,+_+Comedy+_+Mystery,None,,+_+English,TV-14,5.3,2017-05-05,81.0,Handsome: A Netflix Mystery Movie (2017),"2,222","+_+Andrea Seigel,+_+Jeff Garlin",2017
